<a href="https://colab.research.google.com/github/Anushree-B/Lie-detector/blob/main/Lie_detector_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk

In [3]:
tf.__version__

'2.17.0'

In [4]:
df = pd.read_csv("Data/politifact_cleaned.csv")

In [5]:
df.head()

,Politician,Quote,word_count,word_length,adv_count,adj_count,noun_count,verb_count,det_count,sentiment,named_entities,named_entities_count,Truth value
0,5,says that under his presidency the unemployme...,20,5.050000,13.0,13.0,13.0,13.0,13.0,-0.4404,"('american', 'NORP')",1,1.0
1,5,in the u s a person can be married in the m...,24,3.666667,16.0,16.0,16.0,16.0,16.0,0.0000,"('afternoon', 'TIME')",1,1.0
2,2,texas republicans just voted to give a republi...,24,5.416667,21.0,21.0,21.0,21.0,21.0,-0.2500,"('texas', 'GPE'), ('republicans', 'NORP'), ('r...",3,1.0
3,4,the uk has had zero school shootings since it...,17,4.352941,12.0,12.0,12.0,12.0,12.0,-0.3400,"('uk', 'GPE'), ('zero', 'CARDINAL')",2,1.0
4,5,we cut black child poverty in half in bec...,14,4.000000,9.0,9.0,9.0,9.0,9.0,-0.4215,"('half', 'CARDINAL')",1,1.0


# Applying neural network model

In [6]:
# Drop rows with NaN values
df['Truth value'] = df['Truth value'].astype(int)
print(df['Truth value'].isna().sum())
print(df['Truth value'].value_counts())

0
Truth value
3    2405
2     410
1     293
Name: count, dtype: int64


In [7]:
X = df['Quote']
y = df['Truth value']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42,stratify=y)

In [8]:
# Vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [10]:

# Import the machine learning model and train it

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score

models = [LogisticRegression(max_iter=1000), DecisionTreeClassifier(), 
        RandomForestClassifier(n_estimators=200), SVC(), MLPClassifier(),
        MultinomialNB(), BernoulliNB()]

results = []
# Logistic Regression
for model in models:				
		model.fit(X_train_vectorized, y_train)
		y_pred = model.predict(X_test_vectorized) #Type: ignore
		# print("Model: ", model)
		# print("Accuracy: ", accuracy_score(y_test, y_pred))
		# print("F1 Score: ", f1_score(y_test, y_pred, average = 'weighted'))
		# print("Recall: ", recall_score(y_test, y_pred, average = 'weighted'))
		# print("Precision: ", precision_score(y_test, y_pred, average = 'weighted'))
		# print(classification_report(y_test, y_pred))
		results.append([model, accuracy_score(y_test, y_pred), f1_score(y_test, y_pred, average = 'weighted'), recall_score(y_test, y_pred, average = 'weighted'), precision_score(y_test, y_pred, average = 'weighted')])
		print("\n\n")

# Save the best model
best_model = max(results, key = lambda x: x[2])
print(best_model)


d:\Github\Lie-detector\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


d:\Github\Lie-detector\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))














[MLPClassifier(), 0.7218649517684887, 0.6947452631968487, 0.7218649517684887, 0.6746938502075022]


d:\Github\Lie-detector\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Github\Lie-detector\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predicting result with new quote

In [18]:
nltk.download('averaged_perceptron_tagger') #used for tagging words with their parts of speech (POS)
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
X = list(df['Quote'])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42,stratify=y)

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



max_len = 100
max_features = 10000
embed_size = 300
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train)+list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test= tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
X_test = pad_sequences(X_test, padding='post', maxlen=max_len)